In [1]:
import pandas as pd
import numpy as np

# 1 Read Data

In [2]:
# Read the data
title_akas = pd.read_csv('archive/title.akas.tsv/data.tsv', sep='\t', low_memory=False)
title_basics = pd.read_csv('archive/title.basics.tsv/data.tsv', sep='\t', low_memory=False)
title_principals = pd.read_csv('archive/title.principals.tsv/data.tsv', sep='\t', low_memory=False)
title_ratings = pd.read_csv('archive/title.ratings.tsv/data.tsv', sep='\t', low_memory=False)
name_basics = pd.read_csv('archive/name.basics.tsv/data.tsv', sep='\t', low_memory=False)

# 2 Preprocess Data

## 2.1 Preprocess title_basics

In [3]:
# keep only movies
movies = title_basics[title_basics.titleType == 'movie']
movies = movies.drop(columns=['endYear'])
# Replace \N with NaN in the copy of the dataframe
movies.loc[movies.eq('\\N').any(1)] = np.nan
# Drop all rows that contain null or NaN values
movies.dropna(inplace=True)
movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,120,"Adventure,Fantasy"
...,...,...,...,...,...,...,...,...
9699353,tt9916362,movie,Coven,Akelarre,0,2020,92,"Drama,History"
9699437,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,123,Drama
9699478,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,57,Documentary
9699505,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,100,Documentary


In [50]:
# get all the unique genres
genres = set()
for genre in movies.genres:
    genres.update(genre.split(','))

# make genres to a dataframe with a column named 'genre' and a column named 'id'
genres = pd.DataFrame(genres, columns=['genre'])
genres['genre_id'] = genres.index
# save the genres dataframe to a csv file
genres = genres[['genre_id', 'genre']].rename(columns={'genre': 'name'})
genres.to_csv('data/genres.csv', index=False)
genres

,genre_id,name
0,0,Reality-TV
1,1,Biography
2,2,Film-Noir
3,3,Adult
4,4,Horror
5,5,Mystery
6,6,History
7,7,Music
8,8,Documentary
9,9,War


## 2.2 Preprocess title_principals

In [51]:
principal_cast = title_principals[['tconst', 'nconst', 'category', 'characters']]
# keep categories that are 'actor' or 'actress' or 'director'
principal_cast = principal_cast[principal_cast.category.isin(['actor', 'actress', 'director'])]
principal_cast

,tconst,nconst,category,characters
1,tt0000001,nm0005690,director,\N
3,tt0000002,nm0721526,director,\N
5,tt0000003,nm0721526,director,\N
9,tt0000004,nm0721526,director,\N
11,tt0000005,nm0443482,actor,"[""Blacksmith""]"
...,...,...,...,...
55174947,tt9916880,nm1483166,actor,"[""Rude Ralph"",""Mischievous Mike"",""Jolly Josh""]"
55174948,tt9916880,nm0254176,actress,"[""Moody Margaret""]"
55174949,tt9916880,nm0286175,actor,"[""Dad"",""Aerobic Al"",""Nasty Nicola""]"
55174950,tt9916880,nm10535738,actress,"[""Horrid Henry""]"


In [ ]:
# get the actors and actresses from the principal_cast dataframe
act_in = principal_cast[principal_cast.category.isin(['actor', 'actress'])]
# replace "\N" with NaN
act_in = act_in.replace('\\N', np.nan)
# turn the characters column to string type
act_in['characters'] = act_in['characters'].str.strip('["]').str.strip('"]')
act_in

In [ ]:
# get the directors from the principal_cast dataframe
direct = principal_cast[principal_cast.category == 'director']
direct = direct.replace('\\N', np.nan)
direct = direct.drop(columns=['characters', 'category'])
direct

## 2.3 Preprocess title_ratings

In [5]:
title_ratings = title_ratings[['tconst', 'averageRating', 'numVotes']]
title_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1959
1,tt0000002,5.8,263
2,tt0000003,6.5,1799
3,tt0000004,5.6,179
4,tt0000005,6.2,2596
...,...,...,...
1291160,tt9916730,8.3,10
1291161,tt9916766,7.0,21
1291162,tt9916778,7.2,36
1291163,tt9916840,8.8,6


## 2.4 Preprocess title_akas

In [6]:
# only preserve the english titles
movies = movies[movies.tconst.isin(title_akas[title_akas.language == 'en'].titleId)]
movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography"
930,tt0000941,movie,Locura de amor,Locura de amor,0,1909,45,Drama
1600,tt0001614,movie,The Four Devils,De fire djævle,0,1911,60,Drama
1773,tt0001790,movie,"Les Misérables, Part 1: Jean Valjean",Les misérables - Époque 1: Jean Valjean,0,1913,60,Drama
2002,tt0002026,movie,Anny - Story of a Prostitute,Anny - en gatepiges roman,0,1912,68,"Drama,Romance"
...,...,...,...,...,...,...,...,...
9698740,tt9914972,movie,Blind Ambition,Blind Ambition,0,2021,96,Documentary
9699120,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,97,"Comedy,Drama,Fantasy"
9699209,tt9916058,movie,"Cali: ayer, hoy y mañana","Cali: ayer, hoy y mañana",0,1995,250,Documentary
9699260,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama


In [7]:
# left merge movies and title_ratings
movies = pd.merge(movies, title_ratings, on='tconst', how='left')
movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,819.0
1,tt0000941,movie,Locura de amor,Locura de amor,0,1909,45,Drama,4.5,24.0
2,tt0001614,movie,The Four Devils,De fire djævle,0,1911,60,Drama,6.2,33.0
3,tt0001790,movie,"Les Misérables, Part 1: Jean Valjean",Les misérables - Époque 1: Jean Valjean,0,1913,60,Drama,6.2,51.0
4,tt0002026,movie,Anny - Story of a Prostitute,Anny - en gatepiges roman,0,1912,68,"Drama,Romance",4.5,14.0
...,...,...,...,...,...,...,...,...,...,...
124832,tt9914972,movie,Blind Ambition,Blind Ambition,0,2021,96,Documentary,7.1,256.0
124833,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,97,"Comedy,Drama,Fantasy",7.3,8.0
124834,tt9916058,movie,"Cali: ayer, hoy y mañana","Cali: ayer, hoy y mañana",0,1995,250,Documentary,NaN,NaN
124835,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,51,Drama,7.0,7.0


## 2.5 Preprocess name_basics

In [52]:
crews = name_basics.replace('\\N', np.nan)
# filter out rows whose deathYear is less than birthYear
crews = crews[(crews.deathYear >= crews.birthYear) | (crews.deathYear.isnull()) | (crews.birthYear.isnull())]
# filter out null birthYear
crews = crews[crews.birthYear.notnull()]
# only preserve the cast in the movies dataframe
principal_cast = principal_cast[principal_cast.tconst.isin(movies.tconst)]
# preserve the crews in the principal_cast dataframe
crews = crews[crews.nconst.isin(principal_cast.nconst)]
# preserve rows whose nconst is in act_in or direct
subset_crews = crews[crews.nconst.isin(act_in.nconst) | crews.nconst.isin(direct.nconst)]
subset_crews.drop(columns=['knownForTitles'], inplace=True)
subset_crews

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0053137,tt0050419,tt0045537,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0037382,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934,NaN,"actress,soundtrack,music_department","tt0054452,tt0057345,tt0056404,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0072562,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0050976,tt0050986,tt0083922"
...,...,...,...,...,...,...
12378387,nm9990866,Glafira Golubeva,2004,NaN,actress,"tt23024312,tt8006374,tt15488646,tt13926936"
12378503,nm9991000,Divyansha Kaushik,1997,NaN,actress,"tt15028848,tt15309808,tt8737614,tt9872558"
12378784,nm9991365,Benjamin Dino,1996,NaN,actor,"tt14799508,tt18265188,tt5855772"
12379520,nm9992452,Yuki Katayama,1996,NaN,actress,"tt6340500,tt8185442,tt22775702,tt10344670"


In [53]:
# split the knownForTitles column into a list and combine with the nconst column in a new dataframe
known_for_titles = crews[['nconst', 'knownForTitles']].dropna()
known_for_titles['knownForTitles'] = known_for_titles['knownForTitles'].str.split(',')
known_for_titles = known_for_titles.explode('knownForTitles')
# preserve the known_for_titles in the movies dataframe
known_for_titles = known_for_titles[known_for_titles.knownForTitles.isin(movies.tconst)]
known_for_titles


,nconst,knownForTitles
0,nm0000001,tt0053137
0,nm0000001,tt0050419
0,nm0000001,tt0072308
1,nm0000002,tt0038355
1,nm0000002,tt0037382
...,...,...
12379520,nm9992452,tt8185442
12379520,nm9992452,tt22775702
12380554,nm9993693,tt12856788
12380554,nm9993693,tt13847502


# 3.Save the Data

In [67]:
# movies.drop(columns=['genres', 'numVotes'], inplace=True)
movies.to_csv('data/movies.csv', index=False)

In [65]:
subset_crews.to_csv('data/crews.csv', index=False)

In [69]:
known_for_titles.to_csv('data/known_for_titles.csv', index=False)

In [70]:
act_in.to_csv('data/act_in.csv', index=False)
direct.to_csv('data/direct.csv', index=False)